In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import EarlyStopping

from sklearn.metrics import fbeta_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tqdm import tqdm

from utils.file import makedirs
from utils.recorder import record_model_medata, record_model_scores
from utils.loader import load_training_set, load_test_set
from utils.f2thresholdfinder import *
from utils.imagegen import *
from utils.models import *
from utils.custommetrics import *

from datetime import datetime
import time
import configparser
import json
import sys

Using Theano backend.
Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
c:\users\me\appdata\local\temp\try_flags_fbdhsp.c:4:19: fatal error: cudnn.h: No such file or directory
compilation terminated.

Mapped name None to device cuda: GeForce GTX 1060 6GB (0000:01:00.0)


In [2]:
timestr = time.strftime("%Y%m%d-%H%M%S")
start_time = datetime.now()

In [3]:
config_file = 'cfg/default.cfg'

# e.g. > python aggregate_model.py cfg/3.cfg
if len(sys.argv) > 1 and '.cfg' in sys.argv[1]:
    config_file = sys.argv[1]

print('reading configurations from config file: {}'.format(config_file))

settings = configparser.ConfigParser()
settings.read(config_file)
data_dir = settings.get('data', 'data_dir')

df_train = pd.read_csv(data_dir + 'train_v2.csv')
model_filename = 'aggregate_model_'+ timestr +'.h5'
model_filepath = data_dir + 'models/' + model_filename
sample_submission_filepath = data_dir + 'sample_submission_v2.csv'
number_of_samples = len(df_train.index)
print('total number of training samples: {}'.format(number_of_samples))

# WARNING: keras allow either 1, 3, or 4 channels per pixel. Other numbers not allowed.
data_mask_label = np.array(['R', 'G', 'B', 'NDVI', 'NDWI', 'NIR'])
#print(settings.get('data', 'data_mask'))
data_mask_list = json.loads(settings.get('data', 'data_mask'))

data_mask = ma.make_mask(data_mask_list)
print(data_mask)

num_channels = np.sum(data_mask)
need_norm_stats = False

model_id = settings.get('model', 'model_id')
print('model: {}'.format(model_id))

# TODO understand the implications of this. this number usually does not include augmented images but:
# +0.01 F2 score improvement when number_of_samples * 3
num_samples_per_epoch = number_of_samples # number_of_samples * 3 

# default to 64
rescaled_dim = 64
if settings.has_option('data', 'rescaled_dim'):
    rescaled_dim = settings.getint('data', 'rescaled_dim')
print('rescaled dimension: {}'.format(rescaled_dim))

# Per Keras FAQ, one epoch is defined as an arbitrary cutoff that is one pass over the entire training set
number_epoch = settings.getint('model', 'number_epoch')

# a batch results in exactly one update to the model.
# batch_size is limited by model size and GPU memory
batch_size = settings.getint('model', 'batch_size') 
print('batch size: {}'.format(batch_size))

classifier_threshold = 0.2

split = int(number_of_samples * 0.80)  # TODO we may want to increase to 0.90 eventually
number_validations = number_of_samples - split

reading configurations from config file: cfg/9.cfg


C:\Users\Me\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.


total number of training samples: 40479
[ True  True  True  True False False]
model: JAGG_2
rescaled dimension: 128
batch size: 256


In [6]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

print(labels)
print(len(labels))

['slash_burn', 'clear', 'blooming', 'primary', 'cloudy', 'conventional_mine', 'water', 'haze', 'cultivation', 'partly_cloudy', 'artisinal_mine', 'habitation', 'bare_ground', 'blow_down', 'agriculture', 'road', 'selective_logging']
17


In [7]:
label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

In [8]:
x_train, y_train = load_training_set(df_train, rescaled_dim)
print(x_train.shape)
print(y_train.shape)

(40479L, 64L, 64L, 6L)
(40479L, 17L)


In [9]:
x_train = x_train[:, :, :, data_mask]

x_train = x_train.transpose(0,3,1,2)  # https://github.com/fchollet/keras/issues/2681
print(x_train.shape)

(40479L, 3L, 64L, 64L)


In [10]:
# shuffle the samples because 
# 1) the original samples may not be randomized & 
# 2) to avoid the possiblility of overfitting the validation data while we tune the model
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train, random_state=0)

x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

In [11]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(32383L, 3L, 64L, 64L)
(32383L, 17L)
(8096L, 3L, 64L, 64L)
(8096L, 17L)


In [12]:
image_generator = ScaledDown() # NormalizedByFeature() offers seemingly no improvement but cost 30 min more to run.

In [13]:
# this is the augmentation configuration we will use for training
# TODO augment with random rotations for rare classes
train_datagen = image_generator.getTrainGenenerator()

In [14]:
if (need_norm_stats):
    # need to compute internal stats like featurewise std and zca whitening
    train_datagen.fit(x_train)

In [15]:
train_generator = train_datagen.flow(
        x_train, 
        y_train, 
        batch_size=batch_size,
        shuffle=True) 

In [16]:
validation_datagen = image_generator.getValidationGenenerator()

In [17]:
# workaround to provide your own stats: 
# http://stackoverflow.com/questions/41855512/how-does-data-normalization-work-in-keras-during-prediction/43069409#43069409
if (need_norm_stats):
    # need to compute internal stats like featurewise std and zca whitening
    validation_datagen.fit(x_valid)

In [18]:
validation_generator = validation_datagen.flow(
        x_valid,
        y_valid,
        batch_size=batch_size,
        shuffle=False)

In [19]:
model = get_model(model_id, num_channels, rescaled_dim, rescaled_dim)

# TODO 
# Use custom loss function to optimize F2 score.
# https://github.com/fchollet/keras/issues/369
# https://github.com/fchollet/keras/blob/master/keras/losses.py
model.compile(loss='binary_crossentropy', # Is this the best loss function?
              optimizer='adam',
              metrics=['accuracy', 'recall', 'precision'])

In [20]:
# BUG when resuming training, the learning rate need to be decreased.
# let's load an existing trained model and continue training more epoch gives 0.01 improvement in LB score.
# model = load_model(data_dir + 'models/aggregate_model_20170507-124128.h5') # 0.86
# model = load_model(data_dir + 'models/aggregate_model_20170507-184232.h5') # 0.87
# model = load_model(data_dir + 'models/aggregate_model_20170511-133235.h5')
# model = load_model(data_dir + 'models/aggregate_model_20170515-062741.h5')
#number_epoch = 2

In [21]:
# Ran into MemoryError when training DAGG_2 with 4 channels at epoch 50.
# To try to get reduce memory usage, we are limited the number of samples and batch_size

validation_num_samples = min(1280, number_of_samples - split)
x_valid_f2 = x_valid[:validation_num_samples]
y_valid_f2 = y_valid[:validation_num_samples]

# Note: threshold is fixed (not optimized per label)
def compute_f2_measure(l_model):    
    val_generator_f2 = validation_datagen.flow(
        x_valid_f2,
        y_valid_f2,
        batch_size=128,
        shuffle=False)
    raw_pred = l_model.predict_generator(val_generator_f2, validation_num_samples)
    thresholded_pred = (np.array(raw_pred) > classifier_threshold).astype(int)
    l_f2_score = fbeta_score(y_valid_f2, thresholded_pred, beta=2, average='samples')
    return l_f2_score
    
class F2_Validation(k.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.f2_measures = []
    def on_epoch_end(self, epoch, logs={}):
        self.f2_measures.append(compute_f2_measure(self.model))

f2_score_val = F2_Validation()

In [22]:
# stop overfitting on training data
early_stop = EarlyStopping(monitor='val_loss',patience=3, min_delta=0, verbose=0, mode='auto')  # TODO patience and min_delta

In [23]:
training_start_time = datetime.now()
# fits the model on batches with real-time data augmentation:
history = model.fit_generator(train_generator,
                    samples_per_epoch=num_samples_per_epoch,
                    nb_epoch=number_epoch,
                    validation_data=validation_generator,
                    nb_val_samples=number_validations,
                    callbacks=[f2_score_val, early_stop])

model.save(model_filepath)  # always save your model and weights after training or during training
time_spent_trianing = datetime.now() - training_start_time

print('model training complete')

Epoch 1/2
40063/40479 [============================>.] - ETA: 128s - loss: 0.6963 - acc: 0.4485 - recall: 0.6230 - precision: 0.1771 - ETA: 98s - loss: 0.6924 - acc: 0.5192 - recall: 0.5967 - precision: 0.1961  - ETA: 87s - loss: 0.6888 - acc: 0.5578 - recall: 0.5945 - precision: 0.2141 - ETA: 81s - loss: 0.6843 - acc: 0.5815 - recall: 0.5918 - precision: 0.2267 - ETA: 77s - loss: 0.6785 - acc: 0.5995 - recall: 0.5889 - precision: 0.2366

C:\Users\Me\Anaconda2\lib\site-packages\keras\keras\engine\training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


40575/40479 [==============================] - 84s - loss: 0.3943 - acc: 0.8344 - recall: 0.5468 - precision: 0.5726 - val_loss: 0.2537 - val_acc: 0.9045 - val_recall: 0.5658 - val_precision: 0.8112
Epoch 2/2
40575/40479 [==============================] - ETA: 33s - loss: 0.3042 - acc: 0.8871 - recall: 0.5525 - precision: 0.7086 - ETA: 31s - loss: 0.3012 - acc: 0.8880 - recall: 0.5534 - precision: 0.7213 - ETA: 29s - loss: 0.2995 - acc: 0.8896 - recall: 0.5557 - precision: 0.7297 - ETA: 29s - loss: 0.3002 - acc: 0.8887 - recall: 0.5556 - precision: 0.7318

In [ ]:
print(model.summary())

In [24]:

# model = load_model(data_dir + 'models/aggregate_model_20170517-062305.h5')

In [40]:
# use the validation data to compute some stats which tell us how the model is performing on the validation data set.
val_generator_score_board = validation_datagen.flow(
    x_valid,
    y_valid,
    batch_size=batch_size,
    shuffle=False)
p_valid = model.predict_generator(val_generator_score_board, number_validations)

#print(y_valid)
#print(p_valid)

optimized_thresholds = f2_optimized_thresholds(y_valid, np.array(p_valid))

y_predictions = (np.array(p_valid) > optimized_thresholds).astype(int)
#print(y_predictions)

# F2 score, which gives twice the weight to recall emphasising recall higher than precision
# 'samples' is what the evaluation criteria is for the contest
f2_score = fbeta_score(y_valid, y_predictions, beta=2, average='samples')
print('>>>> Overall F2 score over validation set using samples averaging ' , f2_score)


label:0 threshold:0.1 score:0.656204050916
label:1 threshold:0.29 score:0.671182269946
label:2 threshold:0.09 score:0.671182269946
label:3 threshold:0.71 score:0.671185210846
label:4 threshold:0.12 score:0.709070637554
label:5 threshold:0.14 score:0.709070637554
label:6 threshold:0.19 score:0.722336116179
label:7 threshold:0.12 score:0.730768442132
label:8 threshold:0.21 score:0.730768442132
label:9 threshold:0.17 score:0.749661808353
label:10 threshold:0.09 score:0.749661808353
label:11 threshold:0.16 score:0.754054508148
label:12 threshold:0.13 score:0.754138247542
label:13 threshold:0.14 score:0.754138247542
label:14 threshold:0.21 score:0.766468043362
label:15 threshold:0.16 score:0.778192153382
label:16 threshold:0.09 score:0.778192153382
('>>>> Overall F2 score over validation set using samples averaging ', 0.77819215338205638)


In [26]:
threshold_df = pd.DataFrame({'label':labels, 
                             'optimized_threshold':optimized_thresholds})
print(threshold_df)

                label  optimized_threshold
0          slash_burn                 0.10
1               clear                 0.29
2            blooming                 0.09
3             primary                 0.71
4              cloudy                 0.12
5   conventional_mine                 0.14
6               water                 0.19
7                haze                 0.12
8         cultivation                 0.21
9       partly_cloudy                 0.17
10     artisinal_mine                 0.09
11         habitation                 0.16
12        bare_ground                 0.13
13          blow_down                 0.14
14        agriculture                 0.21
15               road                 0.16
16  selective_logging                 0.09


In [33]:
precision_l, recall_l, f2_score_l = calculate_stats_for_prediction(y_valid, y_predictions)

count_stats_df = pd.DataFrame({
    'label': labels, 
    'true_sum': np.sum(y_valid, axis=0),
    'predict_sum': np.sum(y_predictions, axis=0),
    'f2': f2_score_l,
    'recall': recall_l,
    'precision': precision_l
})

# reordering the columns for easier reading
count_stats_df = count_stats_df[['label', 'f2', 'recall', 'precision', 'true_sum', 'predict_sum']]
print(count_stats_df)

                label    f2  recall  precision  true_sum  predict_sum
0          slash_burn  0.50    0.50       0.50        44            0
1               clear  0.61    0.62       0.86      5684         7505
2            blooming  0.50    0.50       0.50        66            0
3             primary  0.49    0.50       0.96      7451         8095
4              cloudy  0.77    0.91       0.64       452         1504
5   conventional_mine  0.50    0.50       0.50        22            0
6               water  0.60    0.62       0.58      1513         2788
7                haze  0.69    0.82       0.61       527         1918
8         cultivation  0.49    0.50       0.45       874            0
9       partly_cloudy  0.52    0.62       0.57      1433         4637
10     artisinal_mine  0.50    0.50       0.50        70            0
11         habitation  0.68    0.71       0.64       732         1166
12        bare_ground  0.57    0.57       0.57       170          167
13          blow_dow

In [ ]:
filtered_data_mask_label = data_mask_label[data_mask]

record_model_scores(model_filename, 
                    model_id, 
                    history, 
                    f2_score, 
                    time_spent_trianing, 
                    num_channels,
                    config_file,
                    np.array_str(filtered_data_mask_label))

In [23]:
figures_dir = 'figures/' + model_id
makedirs(figures_dir)

# list all data in history
print('training history stats:')
print(history.history.keys())

# summarize history for f2 score
fig = plt.figure(figsize=(15, 10))
subplot0 = fig.add_subplot(231)
if hasattr(f2_score_val, 'f2_measures'):
    subplot0.plot(f2_score_val.f2_measures)
subplot0.set_title('f2 score')
subplot0.set_ylabel('f2 score')
subplot0.set_xlabel('epoch')
subplot0.legend(['val'], loc='upper left')

# summarize history for recall
subplot3 = fig.add_subplot(232)
subplot3.plot(history.history['recall'])
subplot3.plot(history.history['val_recall'])
subplot3.set_title('recall')
subplot3.set_ylabel('recall')
subplot3.set_xlabel('epoch')
subplot3.legend(['train', 'val'], loc='upper left')

# summarize history for precision
subplot2 = fig.add_subplot(233)
subplot2.plot(history.history['precision'])
subplot2.plot(history.history['val_precision'])
subplot2.set_title('precision')
subplot2.set_ylabel('precision')
subplot2.set_xlabel('epoch')
subplot2.legend(['train', 'val'], loc='upper left')

# summarize history for accuracy
subplot1 = fig.add_subplot(234)
subplot1.plot(history.history['acc'])
subplot1.plot(history.history['val_acc'])
subplot1.set_title('accuracy')
subplot1.set_ylabel('accuracy')
subplot1.set_xlabel('epoch')
subplot1.legend(['train', 'val'], loc='upper left')

# summarize history for loss
subplot4 = fig.add_subplot(235)
subplot4.plot(history.history['loss'])
subplot4.plot(history.history['val_loss'])
subplot4.set_title('model loss')
subplot4.set_ylabel('loss')
subplot4.set_xlabel('epoch')
subplot4.legend(['train', 'val'], loc='upper left')

fig.savefig(figures_dir + '/stats_' + timestr + '.png')
#plt.show()

training history stats:
['acc', 'loss', 'recall', 'precision', 'val_acc', 'val_recall', 'val_precision', 'val_loss']


In [24]:
#model = load_model(model_filepath)

In [27]:
testset_dir = data_dir + 'test'

df_test_list = pd.read_csv(sample_submission_filepath)

x_test = load_test_set(df_test_list, rescaled_dim)

x_test = x_test[:, :, :, data_mask]

In [28]:
#x_test = np.array(x_test, np.uint8)
print(x_test.shape)
x_test = x_test.transpose(0,3,1,2)  # https://github.com/fchollet/keras/issues/2681
print(x_test.shape)

(61191L, 64L, 64L, 3L)
(61191L, 3L, 64L, 64L)


In [29]:
# this is the configuration we will use for testing:
testset_datagen = image_generator.getTestGenenerator()

if (need_norm_stats):
    # need to compute internal stats like featurewise std and zca whitening
    testset_datagen.fit(x_test)

In [30]:
testset_generator = testset_datagen.flow(
    x_test,
    y=None,
    batch_size=batch_size,
    shuffle=False)
    
# ??? There may be a bug below that casues LB score to be 0.5-0.6
# testset_generator = testset_datagen.flow_from_directory(
#         testset_dir,
#         target_size=(rescaled_dim, rescaled_dim),
#         batch_size=batch_size,
#         class_mode=None,
#         shuffle=False)

In [31]:
#from keras.models import load_model
# model = load_model(data_dir + 'models/aggregate_model_20170507-184232.h5')
# model = load_model(data_dir + 'models/aggregate_model_20170509-215809.h5')
# model = load_model(data_dir + 'models/aggregate_model_20170511-001322.h5')
# model = load_model(data_dir + 'models/aggregate_model_20170511-150149.h5')

In [32]:
# run predictions on test set
testset_predict = model.predict_generator(testset_generator, x_test.shape[0]) # number of test samples

y_testset_predictions = (np.array(testset_predict) > optimized_thresholds).astype(int)

result = pd.DataFrame(y_testset_predictions, columns = labels)

preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.ix[[i]]
    a = a.transpose()
    a = a.loc[a[i] == 1]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

df_test = pd.read_csv(sample_submission_filepath)
df_test['tags'] = preds
df_test
print('done')

100%|██████████████████████████████████████████████████████████████████████████| 61191/61191 [00:57<00:00, 1062.77it/s]


done


In [33]:
#test code
# nums_ones = np.ones((1, 17))
# nums_zeros = np.zeros((1, 17))
# haha = np.array([[1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]])

# y_testset_predictions = haha
# result = pd.DataFrame(y_testset_predictions, columns = labels)

# preds = []
# for i in tqdm(range(result.shape[0]), miniters=1000):
#     a = result.ix[[i]]
#     #print(a)
#     a = a.transpose()
#     print(a)
#     a = a.loc[a[i] == 1]
#     print(a)
#     ' '.join(list(a.index))
#     preds.append(' '.join(list(a.index)))
    
# print(preds)

In [34]:
df_test.to_csv(data_dir + 'my_submissions/submission_' + timestr + '.csv', index=False)

In [35]:
total_exec_time = datetime.now() - start_time
print ('time spent to complete execution: {}'.format(total_exec_time))

time spent to complete execution: 0:05:27.612000
